# *Grapevine Disease - CNN Model*

*Bu projede, üzüm yaprağı görüntülerini analiz ederek dört farklı durumu sınıflandıran bir **CNN Modeli** geliştireceğiz. Modeli kaydedip Streamlit uygulamasına çevireceğiz ve HugginFace de çalışır hale getireceğiz.*

### *İmport Libraries*

In [2]:
import os                  # Dosya ve dizin işlemleri için
import random              # Rastgele seçimler yapmak için
import warnings            # Uyarı mesajlarını yönetmek için
warnings.filterwarnings('ignore') # Tüm uyarı mesajlarını görmezden gelmek için filterwarnings ile 'ignore' ayarını yapıyoruz

import numpy as np         # Sayısal hesaplamalar ve matris işlemleri
import pandas as pd        # Veri analizi ve tablo işlemleri
import cv2                 # Görüntü işleme ve video/frame işlemleri
import kagglehub           # Kaggle veri setlerini kolayca çekmek için
import random

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split  # Veriyi eğitim/test setlerine ayırmak için

from tensorflow.keras.models import Model            # Keras Functional API model tanımı
from tensorflow.keras.layers import Flatten, Dense   # Katmanlar: Flatten ve tam bağlantılı Dense
from tensorflow.keras.applications import VGG16      # Önceden eğitilmiş VGG16 modeli
from tensorflow.keras.preprocessing.image import ImageDataGenerator


### *İmport Data*

In [4]:
# Sınıf etiketlerini bir liste halinde tanımlıyoruz
# 'Black Rot' -> Black Rot örnekleri
# 'ESCA' -> ESCA örnekleri
# 'Healthy' -> Healthy örnekleri
# 'Leaf Blight' -> Leaf Blight örnekleri
labels = ['Black Rot', 'ESCA', 'Healthy', 'Leaf Blight']

# Görüntülerin bulunduğu ana dizin yolunu belirtiyoruz
# Download latest version
path = kagglehub.dataset_download("rm1000/augmented-grape-disease-detection-dataset")

print("Path to dataset files:", path)


Path to dataset files: /root/.cache/kagglehub/datasets/rm1000/augmented-grape-disease-detection-dataset/versions/1


In [5]:
img_path = os.path.join(path, "Final Training Data/")
os.listdir(img_path)

['Healthy', 'Black Rot', 'Leaf Blight', 'ESCA']

### *CNN with Transfer Learning*

In [6]:
# Modelin girdi boyutu
img_width, img_height = 224, 224

# --- Veri hazırlama ---
train_datagen = ImageDataGenerator(rescale=1/255, validation_split=0.20)

train_datagenerator = train_datagen.flow_from_directory(
    directory=img_path,
    target_size=(img_width, img_height),
    class_mode='categorical',
    subset='training'
)

test_datagenerator = train_datagen.flow_from_directory(
    directory=img_path,
    target_size=(img_width, img_height),
    class_mode='categorical',
    subset='validation'
)

# --- Transfer Learning: VGG16 base ---
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
base_model.trainable = False  # Tüm katmanları dondur

# --- Functional API ile üst katmanlar ---
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(4, activation='softmax')(x)  # 5 sınıf

# --- Model tanımı ---
model = Model(inputs=base_model.input, outputs=outputs)

# --- Compile ---
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# --- Eğitim ---
model.fit(
    train_datagenerator,
    epochs=10,
    validation_data=test_datagenerator
)

Found 9600 images belonging to 4 classes.
Found 2400 images belonging to 4 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 74s 199ms/step - accuracy: 0.7996 - loss: 1.8763 - val_accuracy: 0.9471 - val_loss: 0.1774
Epoch 2/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 60s 199ms/step - accuracy: 0.9803 - loss: 0.0531 - val_accuracy: 0.9558 - val_loss: 0.1087
Epoch 3/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 61s 203ms/step - accuracy: 0.9807 - loss: 0.0496 - val_accuracy: 0.9588 - val_loss: 0.1403
Epoch 4/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 62s 205ms/step - accuracy: 0.9955 - loss: 0.0140 - val_accuracy: 0.9671 - val_loss: 0.0807
Epoch 5/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 62s 208ms/step - accuracy: 0.9869 - loss: 0.0360 - val_accuracy: 0.9750 - val_loss: 0.0917
Epoch 6/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 62s 207ms/step - accuracy: 0.9854 - loss: 0.0417 - val_accuracy: 0.9683 - val_loss: 0.0902
Epoch 7/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 62s 207ms/step - accuracy: 0.9950 - los

### *Save the Learning Model*

In [8]:
model.save("grapevine_disease_model_TL.keras")  # Streamlit ve Hugging Face uyumlu

*Bu projede üzüm yaprağı görüntülerini analiz ederek dört farklı durumu sınıflandıran bir CNN Modeli geliştirdik.* <br>
*Datayı kaggle üzerinden çektik.* <br>
*Kendi datamızı önceden eğitilmiş **VGG16** modeli ile birleştirip geliştirdik.* <br>
*%99.68 başarı elde ettik.*

In [10]:
# Test/validation setinden bir batch (küme) görüntü ve etiketleri al
val_images, val_labels = next(test_datagenerator)

# Modeli kullanarak bu görüntüler için tahmin yap
preds = model.predict(val_images)

# One-hot encoding şeklindeki gerçek etiketleri sınıf indekslerine çevir
y_true = np.argmax(val_labels, axis=1)

# Tahmin edilen olasılıkları sınıf indekslerine çevir
y_pred = np.argmax(preds, axis=1)

# Sınıf bazlı doğruluk, precision, recall ve F1-score'u yazdır
# target_names parametresi ile her sınıfın adı gösterilir
print(classification_report(y_true, y_pred, target_names=labels))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step
              precision    recall  f1-score   support

   Black Rot       0.90      1.00      0.95         9
        ESCA       1.00      0.80      0.89         5
     Healthy       1.00      1.00      1.00        10
 Leaf Blight       1.00      1.00      1.00         8

    accuracy                           0.97        32
   macro avg       0.97      0.95      0.96        32
weighted avg       0.97      0.97      0.97        32

